In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 1.1 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705386 sha256=2f585fe750eaa3266afcde8adab07c6f14db62fb524189110d91c650e017472d
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank
import scipy
import time
import random
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import mean_squared_error,mean_absolute_error,roc_auc_score
from math import sqrt
from sklearn import preprocessing
from sklearn.metrics import recall_score,precision_score
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
import pickle
import math

In [5]:
final_dataset=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/nan_clean_dataset.csv")
final_dataset['date'] =pd.to_datetime(final_dataset.date)
final_dataset.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'user_id', 'business_id', 'review_id',
       'rating', 'date', 'useful_x', 'funny_x', 'cool_x', 'text', 'name_x',
       'address', 'city', 'state', 'postal_code', 'latitude', 'longitude',
       'stars', 'review_count_x', 'category', 'RestaurantsAttire',
       'RestaurantsTakeOut', 'Alcohol', 'RestaurantsDelivery', 'WiFi',
       'GoodForKids', 'RestaurantsReservations', 'HasTV', 'Caters',
       'RestaurantsPriceRange2', 'BikeParking', 'BusinessAcceptsCreditCards',
       'RestaurantsGoodForGroups', 'NoiseLevel', 'OutdoorSeating', 'name_y',
       'review_count_y', 'yelping_since', 'useful_y', 'funny_y', 'cool_y',
       'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [6]:
df_sample2 = final_dataset.drop(columns=['Unnamed: 0','Unnamed: 0.1','elite','address','state','postal_code','latitude','longitude'])


In [7]:
users_=final_dataset.user_id.value_counts()
active_users = users_.loc[users_ >5].index.to_list()
active = pd.DataFrame({"user_id": active_users})
df_sample2 = pd.merge(left = final_dataset,right= active,left_on='user_id',right_on='user_id')

In [8]:
df=df_sample2.groupby(['user_id']).date.apply(lambda x: max(x)).reset_index()
df_test = pd.merge(df_sample2,df, how='inner',on=['user_id','date'])
df_train=df_sample2[~df_sample2.isin(df_test.to_dict('l')).all(1)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# read in cleaned review data
review_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/ratings1.csv')

In [10]:
business_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/business3.csv',index_col=0)
user_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/users_tag.csv',index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
n_users = review_df.user_id.unique().shape[0]
n_items = review_df.business_id.unique().shape[0]
print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
#print('Sparsity: {:4.3f}%'.format(float(review_df.shape[0]) / float(n_users*n_items) * 100))

Number of users: 4001
Number of models: 1064


In [12]:
review_df.cool = pd.Series([math.log(x+1) for x in review_df.cool])
review_df.useful = pd.Series([math.log(x+1) for x in review_df.useful])
review_df.funny = pd.Series([math.log(x+1) for x in review_df.funny])

In [13]:
# clean user skewness
user_df.review_count = pd.Series([math.log(x+1) for x in user_df.review_count])
user_df.useful =  pd.Series([math.log(x+1) for x in user_df.useful])

#cleam business skewness
business_df.review_count =  pd.Series([math.log(x+1) for x in business_df.review_count])

In [14]:
user_df=user_df[['average_stars','compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
        'funny', 'review_count', 'useful','is_elite', 'year','user_id']]

Building dataset


In [15]:
#model establishment
dataset = Dataset()
dataset.fit(review_df.user_id,review_df.business_id)
type(dataset)
num_users, num_items = dataset.interactions_shape()



In [16]:
dataset.fit_partial(items=business_df.business_id,
                    item_features=['stars'])
dataset.fit_partial(items=business_df.business_id,
                    item_features=['review_count'])
tar_cols = [x for x in business_df.columns[26:]]

dataset.fit_partial(items = business_df.business_id,
                   item_features = tar_cols)                                    
                                                

user_cols = [x for x in user_df[['compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 
         'review_count', 'useful','is_elite']]]
dataset.fit_partial(users=user_df.user_id,
                    user_features = user_cols)

In [17]:
#build interaction
(interactions, weights) = dataset.build_interactions([(x['user_id'],
                                                       x['business_id'],
                                                       x['rating']) for index,x in review_df.iterrows()])

print(repr(interactions))

<43234x20016 sparse matrix of type '<class 'numpy.int32'>'
	with 26183 stored elements in COOrdinate format>


In [18]:
#building item and user matrix
def build_dict(df,tar_cols,val_list):
    rst = {}
    for col in tar_cols:
        rst[col] = df[col]
    sum_val = sum(list(rst.values())) # get sum of all the tfidf values
    
    if(sum_val == 0):
        return rst
    else:
        
        w = (2-sum(val_list))/sum_val # weight for each tag to be able to sum to 1
        for key,value in rst.items():
            rst[key] = value * w
    return rst

def user_build_dict(df,tar_cols,val_list):
    rst = {}
    for col in tar_cols:
        rst[col] = df[col]
    sum_val = sum(list(rst.values())) # get sum of all the tfidf values
    
    if(sum_val == 0):
        return rst
    else:
        w = (2-sum(val_list))/sum_val # weight for each tag to be able to sum to 1
        for key,value in rst.items():
            rst[key] = value * w
    return rst

# get max of each column to regularize value to [0,1]
max_star = max(business_df.stars)
max_b_rc = max(business_df.review_count)
print('max_b_rc')
print(max_b_rc)

# give CF info weight 0.5, all other 0.5. Then in others, give (star, review count) 0.25 and tags 0.25
item_features = dataset.build_item_features(((x['business_id'], 
                                              {'stars':0.5*x['stars']/max_star,
                                               'review_count':0.5*x['review_count']/max_b_rc,
                                               **build_dict(x,tar_cols,[0.5*x['stars']/max_star,
                                                           0.5*x['review_count']/max_b_rc])})
                                              for index,x in business_df.iterrows()))


# user_features = dataset.build_user_features(((x['user_id'],
#                                              [x['is_elite'],x['year']])
#                                            for index, x in data_users.iterrows()))
max_u_rc = max(user_df.review_count)
max_useful = max(user_df.useful)
user_features = dataset.build_user_features(((x['user_id'],
                                             {'review_count':0.35*x['review_count']/max_u_rc,'is_elite':0.35*int(x['is_elite']),
                                              'useful':0.35*x['useful']/max_useful,
                                             **user_build_dict(x,user_cols,[0.35*x['review_count']/max_u_rc,
                                                                            0.35*int(x['is_elite']),
                                                                            0.35*x['useful']/max_useful])})
                                           for index, x in user_df.iterrows()))

print(repr(item_features))
print(item_features.shape)

print(repr(user_features))
print(user_features.shape)

max_b_rc
9.12543586488574
<20016x20079 sparse matrix of type '<class 'numpy.float32'>'
	with 1281024 stored elements in Compressed Sparse Row format>
(20016, 20079)
<43234x43248 sparse matrix of type '<class 'numpy.float32'>'
	with 606258 stored elements in Compressed Sparse Row format>
(43234, 43248)


In [19]:
seed = 123
from lightfm.cross_validation import random_train_test_split
train,test=random_train_test_split(interactions,test_percentage=0.2,random_state=np.random.RandomState(seed))

print('The dataset has %s users and %s items, '
      'with %s interactions in the test and %s interactions in the training set.'
      % (train.shape[0], train.shape[1], test.getnnz(), train.getnnz()))

train.multiply(test).nnz == 0 # make sure train and test are truly disjoint

The dataset has 43234 users and 20016 items, with 5237 interactions in the test and 20946 interactions in the training set.


False

In [20]:
k=5
NUM_THREADS = 25
NUM_COMPONENTS = 42    
NUM_EPOCHS = 14
ITEM_ALPHA = 0.000256
learning_rate=0.0529
# WARP
model_iii = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA, random_state=seed,
               no_components=NUM_COMPONENTS)

#time it.
%time model_iii = model_iii.fit(train,user_features=user_features,item_features=item_features,epochs=NUM_EPOCHS,num_threads=NUM_THREADS)



# Compute and print the AUC score
train_auc = auc_score(model_iii, train,user_features=user_features,item_features=item_features, num_threads=NUM_THREADS).mean()
print('Hybrid train AUC: %s' % train_auc)

test_auc = auc_score(model_iii, test,user_features=user_features,item_features=item_features,num_threads=NUM_THREADS).mean()
print('Hybrid test AUC: %s' % test_auc)


CPU times: user 18.7 s, sys: 16.9 ms, total: 18.7 s
Wall time: 9.63 s
Hybrid train AUC: 0.99283177
Hybrid test AUC: 0.9852828


In [22]:
import Rec_fx as rf

In [24]:
rf.sample_train_recommendation(model_iii,train,business_df,[600],5,'name',mapping=dataset.mapping()[2],tag='category',
                              user_features = user_features,item_features=item_features)

rf.sample_test_recommendation(model_iii,train,test,business_df,[1],5,'name',mapping=dataset.mapping()[2],
                              train_interactions=train,tag='category',user_features = user_features,item_features=item_features)

**User 600**

**Known positives:**

 Bawarchi Tikka Kabob & Curry House | Indian, Restaurants
 The Roost | Bars, Restaurants, Sandwiches, American (Traditional), Cocktail Bars, Chicken Shop, Nightlife
 Churro | Desserts, Ice Cream & Frozen Yogurt, Food, Food Trucks
 Little Sub Trailer | Restaurants, Sandwiches, Food Stands
 L & C Bubble Tea House | Restaurants, Coffee & Tea, Taiwanese, Food
 Pacific Grille | Restaurants, American (Traditional)
 Café Nordstrom | Cafes, Diners, American (New), Restaurants
 Sugarloop | Desserts, Cafes, Nightlife, Restaurants, Sporting Goods, Shopping, Food
 The Meadow | Cards & Stationery, Ethnic Food, Beer, Wine & Spirits, Shopping, Candy Stores, Arts & Crafts, Beauty & Spas, Florists, Cosmetics & Beauty Supply, Chocolatiers & Shops, Food, Specialty Food, Event Planning & Services, Herbs & Spices, Gift Shops, Flowers & Gifts
 Sciué Italian Bakery Caffé | Breakfast & Brunch, Pizza, Coffee & Tea, Italian, Restaurants, Bakeries, Food, Cafes
 Starbucks | Food, Coffee & Tea, Bakeries
 Popeyes L

**Recommended:**

 Zaika Indian Contemporary Cuisine | Restaurants, Indian
 Nana's Ice Cream Scoop Shop | Food, Ice Cream & Frozen Yogurt
 Ocean King Market | Food, Seafood Markets, International Grocery, Specialty Food, Ethnic Food
 Plumper Pumpkin Patch and Tree Farm | Home Services, Farms, Specialty Food, Local Flavor, Food, Plumbing, Fruits & Veggies, Arts & Entertainment, Attraction Farms, Home Decor, Pumpkin Patches, Home & Garden, Christmas Trees, Farmers Market, Shopping
 Yaletown Liquor Store | Beer, Wine & Spirits, Food


...

**Recommended:**

 Let's Roll Custom Sushi Bar | Restaurants, Canadian (New), Sushi Bars, American (New)
 Nana's Ice Cream Scoop Shop | Food, Ice Cream & Frozen Yogurt
 Brian's Brew | Food, Coffee & Tea
This one clicked
 Taqueria So Mexican | Restaurants, Mexican, Food Trucks, Food, Food Stands
 Jackson's Poultry | Food, Specialty Food, Farmers Market, Butcher, Meat Shops


*k_p: 8*

*precicion at k : 0.2*

----------------------------------------------------------------------
